# Chest Clinic

In [10]:
import pandas as pd
import numpy as np

path1k = r"../sample generator/amostras1000.csv"; 
path10k = r"../sample generator/amostras10000.txt"; 

df_chest1k = pd.read_csv(path1k, header=None)
df_chest1k.columns = ['Visita Asia', 'Fuma', 'Tuberculose', 'Cancer Pulmao', 
'Bronquite', 'Tuberculose ou Cancer', 'RaioX', 
'Dispneia']
df_chest10k = pd.read_csv(path10k, header=None)

### Separação em conjunto de treinamento (70% da base) e teste (30% da base)

In [19]:
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
#X = Variables | Y = Class
X1k, y1k = df_chest1k.iloc[:, :].values, df_chest1k.iloc[:, 7].values
X1k_train, X1k_test, y1k_train, y1k_test = \
        train_test_split(X1k, y1k, test_size=0.3, random_state=0)

X10k, y10k = df_chest10k.iloc[:, :].values, df_chest10k.iloc[:, 7].values
X10k_train, X10k_test, y10k_train, y10k_test = \
        train_test_split(X10k, y10k, test_size=0.3, random_state=0)

      0  1  2  3  4  5  6
7681  0  0  0  0  0  0  1
9031  0  1  0  1  0  1  0
3691  0  1  0  1  1  1  0
202   0  0  0  0  0  0  1
5625  0  0  0  0  1  0  1
298   0  0  0  0  1  0  1
5135  0  0  0  0  1  0  1
6899  0  0  0  0  0  1  0
7258  0  1  0  0  0  0  1
9516  0  1  0  0  1  0  0
6140  0  0  0  0  1  0  1
9082  0  0  0  0  0  0  1
7205  0  1  0  0  1  0  1
2819  0  0  0  0  0  0  1
5282  0  1  0  0  1  0  1
4901  0  0  0  0  0  0  1
8449  0  0  0  0  0  0  1
9949  0  1  0  0  0  0  1
5466  0  1  0  0  1  0  1
4503  0  1  0  0  1  0  0
210   0  1  0  0  1  0  1
7658  0  0  0  0  0  0  1
5964  0  0  0  0  0  0  0
3650  0  1  0  0  1  0  1
5624  0  0  0  0  1  0  1
8728  0  0  0  0  0  0  1
4919  0  0  0  0  0  0  1
2345  0  1  0  0  1  0  1
4162  0  1  0  0  0  0  1
1153  0  1  0  0  1  0  1
...  .. .. .. .. .. .. ..
755   0  0  0  0  1  0  1
8291  0  0  0  0  0  0  1
2496  0  0  0  0  1  0  1
7599  0  0  0  0  0  0  1
1871  0  0  0  0  0  0  1
2046  0  0  0  0  1  0  1
7877  0  1  

KeyError: 'Dispneia'

# <i>Random Forests</i>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

feat_labels = df_chest1k.columns

forest = RandomForestClassifier(n_estimators=10000,
                                criterion='entropy',
                                max_features='sqrt',
                                random_state=0,
                                n_jobs=-1)

forest.fit(X10k_train, y10k_train)
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]

for f in range(X10k_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))

plt.title('Feature Importances')
plt.bar(range(X10k_train.shape[1]), 
        importances[indices],
        color='blue', 
        align='center')

plt.xticks(range(X10k_train.shape[1]), 
           feat_labels[indices], rotation=90)
plt.xlim([-1, X10k_train.shape[1]])
plt.tight_layout()
plt.show()

# <i>Decision Tree</i>

In [ ]:
import graphviz
from sklearn import tree

clf = tree.DecisionTreeClassifier(criterion='entropy',
                                max_features='sqrt',
                                random_state=0)
#Treinamento com base de dados de mil amostras
# clf1k = clf.fit(X1k_train, y1k_train)

#Treinamento com base de dados de dez mil amostras
clf10k = clf.fit(X10k_train, y10k_train)

#Plotagem das importâncias das características da base de dados de dez mil amostras
importancesTree = clf10k.feature_importances_
indices = np.argsort(importancesTree)[::-1]
for f in range(X10k_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importancesTree[indices[f]]))
    
plt.title('Feature Importances')
plt.bar(range(X10k_train.shape[1]), 
        importancesTree[indices],
        color='blue', 
        align='center')

plt.xticks(range(X10k_train.shape[1]), 
           feat_labels[indices], rotation=90)
plt.xlim([-1, X10k_train.shape[1]])
plt.tight_layout()
plt.show()

In [ ]:
#Utilização da biblioteca graphviz para desenhar a árvore de decisão
dot_data1k = tree.export_graphviz(clf1k, proportion=True, feature_names=['Visita Asia', 'Fuma', 'Tuberculose', 'Cancer Pulmao', 'Bronquite', 'Tuberculose ou Cancer', 'RaioX', 'Dispneia'], out_file= None, filled = True) 
dot_data10k = tree.export_graphviz(clf10k, proportion=True, feature_names=['Visita Asia', 'Fuma', 'Tuberculose', 'Cancer Pulmao', 'Bronquite', 'Tuberculose ou Cancer', 'RaioX', 'Dispneia'], out_file= None, filled = True) 

graph1k = graphviz.Source(dot_data1k)
graph10k= graphviz.Source(dot_data10k)

#Exportação da árvore de mil amostras em pdf
graph1k.render('tree1k')
#Exportação da árvore de dez mil amostras em pdf
graph10k.render('tree10k')
#Plotagem da árvore de decisão de dez mil amostras
# graph1k
graph10k
